In [5]:
#import libraries
from time import time, sleep
import random
import urllib
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
import bs4
from bs4 import BeautifulSoup
import re
from IPython.display import clear_output
import os
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import visibility_of_element_located
from selenium.webdriver.common.action_chains import ActionChains

In [6]:
#URL parsing
#Check URL format:
#https://www.pets4homes.co.uk/sale/reptiles/local/
#https://www.pets4homes.co.uk/sale/reptiles/local/birmingham/
#https://www.pets4homes.co.uk/sale/reptiles/local/birmingham/page-11/
#https://www.pets4homes.co.uk/sale/dogs/german-shepherd/
#https://www.pets4homes.co.uk/sale/dogs/german-shepherd/local/page-35/
#https://www.pets4homes.co.uk/sale/dogs/german-shepherd/durham/page-2/
#https://www.pets4homes.co.uk/sale/reptiles/hermann-tortoise/liverpool/
#https://www.pets4homes.co.uk/adoption/reptiles/hermann-tortoise/liverpool/
#https://www.pets4homes.co.uk/stud/reptiles/hermann-tortoise/liverpool/
#https://www.pets4homes.co.uk/sale/dogs/local/aberdeen/
#URL format is kind of pain to work with, probably should scrape by type of listing, type of pet and city separately.
#species or breed would be listed in the dataframe, no need to be scraped individually.
#so the format would be:
#https://www.pets4homes.co.uk/<type of listing>/<type of pets>/local/<city>/<page>
#page don't exist in URL for page 1.

In [7]:
url_template = "https://www.pets4homes.co.uk/{}/{}/local/{}/"
listing_type = ['sale','adoption','stud']
pet_type = ['dogs','cats','reptiles']
cities = ['aberdeen','ayr_st-ives', 'barnsley_wimborne','basingstoke','bath','bedfordshire']
browser = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\fullb\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
C:\Users\fullb\AppData\Local\Temp/ipykernel_20056/1326188593.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


In [9]:
for city in set (cities):
    for pets in set(pet_type):
        for listing in set(listing_type):
            url = url_template.format(listing, pets, city)
            options = webdriver.ChromeOptions()
            browser = webdriver.Chrome(ChromeDriverManager().install(),options = options)
            browser.get(url)
            title = (
                WebDriverWait(driver=browser, timeout=10)
                .until(visibility_of_element_located((By.CSS_SELECTOR, "h1"))).text
            )
            content = browser.page_source
            soup = BeautifulSoup(content, 'lxml')
            print(url)
            sleep(random.randint(10,15))
            
            sleep(random.randint(10,30))



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\fullb\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
C:\Users\fullb\AppData\Local\Temp/ipykernel_20056/3222587674.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(),options = options)


https://www.pets4homes.co.uk/adoption/dogs/local/bath/


C:\Users\fullb\AppData\Local\Temp/ipykernel_20056/3222587674.py:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  target = browser.find_element_by_link_text("Next page")


====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\fullb\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


https://www.pets4homes.co.uk/adoption/reptiles/local/bath/




====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\fullb\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


...



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\fullb\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


https://www.pets4homes.co.uk/adoption/reptiles/local/aberdeen/




====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\fullb\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


https://www.pets4homes.co.uk/adoption/cats/local/aberdeen/


KeyboardInterrupt: 

In [ ]:
url_template = "https://www.pets4homes.co.uk/{}/{}/local/{}/"
for city in set (cities):
    for pets in set(pet_type):
        for listing in set(listing_type):
            url = url_template.format(listing, pets, city)
            browser.get(url)
            content = browser.page_source
            soup = BeautifulSoup(content, 'lxml')
            print(url)